In [1]:
import mysql.connector as mariadb
import csv

In [2]:
with open("./stored_authors/root_names.txt", 'r') as root_out:
    root_names = eval(root_out.read())
with open("./stored_authors/mergees.txt", 'r') as mergees_out:
    mergees = eval(mergees_out.read())

In [3]:
def select_and_print(cursor, command_string):
    """
    Selects entries from database and prints it out to the python interpreter. 
    
    Parameters: 
    cursor         : Sqlite cursor object - the sqlite3 cursor created from .cursor() on a connected sqlite 3 database
    command_string : String               - the entire select SQL command (e.g. "select * from testing123;")
    """
    
    cursor.execute(command_string)

    rows = cursor.fetchall()
    for row in rows:
        print(row.keys())

        string = ""
        for key in row.keys():
            string += str(row[key])
            string += " | "
        # print(str(row['test_1']) + " | " + str(row['test_2']))

        print(string)
        
    # print(rows.keys())
    
    # print(rows[0]['test_1'])
    # print(rows[0]['test_2'])
    print("---------------")

In [7]:
def update_values(cursor, root_id, mergee_id, new_root_name):
    # Updates given, middle, and family names of root_id
    
    if new_root_name != "root":
        string_tokens = new_root_name.split('_')

        root_name_update = "update DBADSORPTION_SANDBOX.authors set given_name = '" + string_tokens[0] + "' where author_id = '" + root_id + "';"
        print(root_name_update) #
#         cursor.execute(root_name_update)
        root_name_update = "update DBADSORPTION_SANDBOX.authors set middle_name = '" + string_tokens[1] + "' where author_id = '" + root_id + "';"
        print(root_name_update) # 
#         cursor.execute(root_name_update)
        root_name_update = "update DBADSORPTION_SANDBOX.authors set family_name = '" + string_tokens[2] + "' where author_id = '" + root_id + "';"
        print(root_name_update) # 

#         cursor.execute(root_name_update)

    # Iterates through mergees and changes each foreign key in biblio_authors to the root_id 
    foreign_change_string = "update DBADSORPTION_SANDBOX.biblio_authors set author_id = '" + root_id + "' where author_id = '" +  mergee_id + "';"
    print(foreign_change_string) #
#     cursor.execute(foreign_change_string)
    delete_string = "DELETE from DBADSORPTION_SANDBOX.authors WHERE author_id = '" + mergee_id + "';"
    print(delete_string) #
#     cursor.execute(delete_string)    

In [8]:
# mariadb_connect = mariadb.connect(user="jfl2@localhost", passwd="jfl2_mysql", database="DBADSORPTION_SANDBOX")
cursor = "placeholder"
already_done_foreign = []
for root, pair in mergees.items():
    
    for person in pair:
        already_done_foreign.append(person)
        
        if root not in already_done_foreign:
            update_values(cursor, root, person, root_names[root])

update DBADSORPTION_SANDBOX.authors set given_name = 'Yves' where author_id = '0000-0001-6082-5862';
update DBADSORPTION_SANDBOX.authors set middle_name = '' where author_id = '0000-0001-6082-5862';
update DBADSORPTION_SANDBOX.authors set family_name = 'Gensterblum' where author_id = '0000-0001-6082-5862';
update DBADSORPTION_SANDBOX.biblio_authors set author_id = '0000-0001-6082-5862' where author_id = 'ec2a8f1baf004250d3bb99e1f69bec1ea50539f6';
DELETE from DBADSORPTION_SANDBOX.authors WHERE author_id = 'ec2a8f1baf004250d3bb99e1f69bec1ea50539f6';
update DBADSORPTION_SANDBOX.authors set given_name = 'Pravas' where author_id = '0000-0001-7998-4492';
update DBADSORPTION_SANDBOX.authors set middle_name = '' where author_id = '0000-0001-7998-4492';
update DBADSORPTION_SANDBOX.authors set family_name = 'Deria' where author_id = '0000-0001-7998-4492';
update DBADSORPTION_SANDBOX.biblio_authors set author_id = '0000-0001-7998-4492' where author_id = '83f0d3c366a39d2a95cb093ada562fd1b99712e7';